In [2]:
pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.3 MB

# import / 라이브러리 호출

In [3]:
import pandas as pd
import random
import os
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

from autogluon.tabular import TabularPredictor, TabularDataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Fixed RandomSeed / 랜덤시드 고정

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(96) # Seed 고정

In [6]:
train = pd.read_csv("/content/drive/MyDrive/Dacon/추석 맞이 추석 선물 수요량 예측 AI 경진대회/open/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Dacon/추석 맞이 추석 선물 수요량 예측 AI 경진대회/open/test.csv")

In [7]:
train.head()

,ID,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형,수요량
0,TRAIN_0000,1,쇼핑몰 15,212000,0,도시 6,지역 1,쇼핑몰 유형 2,명절혼합과일선물세트,28
1,TRAIN_0001,2,쇼핑몰 72,113000,0,도시 21,지역 1,쇼핑몰 유형 1,발효홍삼선물세트,27
2,TRAIN_0002,0,쇼핑몰 15,67000,0,도시 6,지역 1,쇼핑몰 유형 2,실속스팸선물세트,769
3,TRAIN_0003,1,쇼핑몰 13,206000,0,도시 12,지역 3,쇼핑몰 유형 1,자연산프리미엄버섯선물세트,27
4,TRAIN_0004,1,쇼핑몰 65,140000,0,도시 16,지역 2,쇼핑몰 유형 2,자연산새우선물세트,337


# AutoGluon

In [8]:
train.drop("ID", axis = 1, inplace = True)
test.drop("ID", axis = 1, inplace = True)

In [9]:
tune_kwargs = {
    'searcher': 'random',
    'num_trials': 7,
    'scheduler' : 'local'
}

In [10]:
pred = TabularPredictor(label="수요량", eval_metric="rmse", problem_type="regression").fit(train_data=train,
                                                                                          presets=["best_quality"],
                                                                                          hyperparameter_tune_kwargs=tune_kwargs,
                                                                                          num_bag_folds=20,
                                                                                          refit_full=True,
                                                                                          time_limit=43200)

Fitted model: NeuralNetTorch_BAG_L2/9e9ac_00000 ...
	-130.583	 = Validation score   (-root_mean_squared_error)
	381.58s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L2/9e9ac_00001 ...
	-122.7417	 = Validation score   (-root_mean_squared_error)
	472.26s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L2/9e9ac_00002 ...
	-125.0187	 = Validation score   (-root_mean_squared_error)
	571.02s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMLarge_BAG_L2 ... Training model for up to 1768.07s of the 7955.43s of remaining time.
	Fitting 20 child models (S1F1 - S1F20) | Fitting with ParallelLocalFoldFittingStrategy
	-121.8827	 = Validation score   (-root_mean_squared_error)
	415.37s	 = Training   runtime
	2.04s	 = Validation runtime
Completed 1/20 k-fold bagging repeats ...
Fitting model: WeightedEnsemble_L3 ... Training model for up to 1768.07s of the 7526.86s of remaining time.
	-110.7721	 = Validat

In [19]:
print(pred.fit_summary())

*** Summary of fit() ***
Estimated performance of each model:
                                       model   score_val  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0                        WeightedEnsemble_L3 -110.772088     218.392030  26413.166991                0.002310           1.602372            3       True         69
1         NeuralNetFastAI_BAG_L2/a674b_00001 -112.103090     218.388588  24619.508873                0.000530         360.140682            2       True         54
2         NeuralNetFastAI_BAG_L2/a674b_00004 -113.187065     218.388204  24506.324371                0.000146         246.956180            2       True         57
3         NeuralNetFastAI_BAG_L2/a674b_00000 -114.260718     218.388285  24609.197902                0.000227         349.829711            2       True         53
4                          XGBoost_BAG_L2/T2 -114.831340     218.388226  24404.709945                0.000167         

In [11]:
predict = pred.predict(test)

In [14]:
sub = pd.read_csv("/content/drive/MyDrive/Dacon/추석 맞이 추석 선물 수요량 예측 AI 경진대회/open/sample_submission.csv")

In [15]:
sub["수요량"] = predict

In [16]:
sub.to_csv("AutoGluon_ver1.csv", index = False)

In [17]:
from google.colab import files
files.download('AutoGluon_ver1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

public점수: 105.5136772481
private점수: 121.2534742947